<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment03/seq2seq_without_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/lexicons.zip'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [ ]:
train_df = pd.read_csv("/content/lexicons/ta.translit.sampled.train.tsv", sep = '\t', header = None)
train_df = train_df.dropna(axis=0)

val_df = pd.read_csv("/content/lexicons/ta.translit.sampled.dev.tsv", sep = '\t', header = None)
val_df = val_df.dropna(axis=0)

In [ ]:
train_df

,0,1,2
0,ஃபியட்,fiat,2
1,ஃபியட்,phiyat,1
2,ஃபியட்,piyat,1
3,ஃபிரான்ஸ்,firaans,1
4,ஃபிரான்ஸ்,france,2
...,...,...,...
68213,ஹோல்ட்,holtt,1
68214,ஹோல்ட்,hoold,1
68215,ஹோல்ட்,hoolt,1
68216,ஹோல்ட்,hooltt,1


In [ ]:
val_df

,0,1,2
0,ஃபயர்,fire,3
1,ஃபயர்,phayar,1
2,ஃபார்,baar,1
3,ஃபார்,bar,1
4,ஃபார்,far,1
...,...,...,...
6822,ஹோலி,hoolley,1
6823,ஹோலி,hoolli,2
6824,ஹோலி,hoolly,1
6825,ஹோலி,hooly,2


In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for input_text,target_text in zip(train_df[1][0:],train_df[0][0:]):
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

Number of samples: 68215
Number of unique input tokens: 26
Number of unique output tokens: 46
Max sequence length for inputs: 30
Max sequence length for outputs: 26


In [ ]:
# Vectorize the data.
val_input_texts = []
val_target_texts = []

for val_input_text,val_target_text in zip(val_df[1][0:],val_df[0][0:]):
    val_input_texts.append(val_input_text)
    val_target_texts.append(val_target_text)

val_encoder_input_data = np.zeros(
    (len(val_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
val_decoder_input_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
val_decoder_target_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(val_input_text):
        val_encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(val_target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

In [ ]:
latent_dim = 256

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
print(len(val_input_texts))

6827


In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([encoder_input_data, decoder_input_data],
                    decoder_target_data)
)
# Save model
model.save("s2s")